# AI PA1: NQueens
**Group members:**
- Furkan Bulut, 210316011
- İsmet Eren Yurtcu, 210316028
- Gonca Arabacı, 210316067


**Readme**

Give some information about your Python version, development environment, etc.
Pycharm, Python 3.11.5

## Part 1: Class definition

This is the part that you will implement. See the comments in the code and read the assignment description.

In [1]:
#import files if needed
import random
#class definition for NQueens
class NQueens():
    """ class constructor
    initializes the instance attributes N and state """
    def __init__(self, N):
        self.N = N #Detemines number of queen and board size
        self.state = self._set_state() # state cannot be greater than N
        
    """ returns a formatted string
    that represents the instance """        
    def __str__(self):
        return f"N: {self.N}\nstate: {self.state}"

    """ Sets the instance attribute state by displaying 
    a menu to the user. The user either enters the state 
    manually or prompts the system to generate a random state.
    Check if the input state is a valid state. """         
    def _set_state(self):
        print(self.N)
        print("How do you want to set state?")
        print("1. Set state manually")
        print("2. Set state randomly")
        state_choice = int(input("Enter Selection"))
        
        if state_choice == 1:
            print("Enter state: ")
            state = input()
            is_valid = self._is_valid(state)
            if is_valid:
                return state
            else:
                print("Invalid state! Please try again...")
                return  self._set_state()
        elif state_choice == 2:
            state = self.generate_random_state()
            return state
        else:
            print("Invalid Selection! Please input again...")
            return self._set_state()
            

    """ generates and returns a valid random state """
    def generate_random_state(self):
        state = [random.randint(1,self.N) for i in range(self.N)] #state cannot be greater than N and smaller than 1
        return state
    
    """ This is an internal function that takes a state as input
    and return if this is a valid state or not """
    def _is_valid(self,state):
        if len(state) > self.N and len(state)!=self.N : #state length cannot be greater than N
            print("Invalid state input. Error Code 1")
            return False
        elif state.isdigit() == False: #state must be integer
            print("Invalid state input. Error Code 2")
            return False
        elif int(min(state))>1 and int(max(state)) > self.N:
            # the included number of a state cannot be greater than N. Or cannot be smaller than 0 . (123457) N=6 it is not possible
            print("Invalid state input. Error Code 3")
            return False
        else: 
            return True
            

    """ This is the primary function of this class.
    It returns the number of attacking pairs in the given state board.
    """  
    def _count_attacking_pairs(self, state):
        state = [int(s) for s in state]  # Convert integers
        num_pairs = 0
    
        for i in range(self.N): # For each column
            for j in range(i + 1, self.N): # For each column after the current column
                # Column attack check
                if state[i] == state[j]: # If the queens are in the same row
                    num_pairs += 1 # Add 1 to the number of attacking pairs
                    
                # Diagonal attack check (both directions) 
                if abs(state[i] - state[j]) == abs(i - j): # If the queens are in the same diagonal
                    num_pairs += 1 # Add 1 to the number of attacking pairs

        
        print("Number of attacking pairs:", num_pairs) 
        return num_pairs



## Part 2: Testing

Do not change this part. This is the test code.

In [2]:
# This is a test code. You can try with different N values and states.
problem = NQueens(int(input("Input the board size (N)"))) #create NQueens instance
print(problem) #print the description of the problem
print(problem._count_attacking_pairs(problem.state)) #print the total number of attacking pairs in the board

ValueError: invalid literal for int() with base 10: ''

AI PA2

In [72]:
from simpleai.search import SearchProblem, breadth_first, uniform_cost, depth_first, limited_depth_first, iterative_limited_depth_first, greedy, astar
from simpleai.search.viewers import WebViewer
import random
from datetime import datetime

class NQueens(SearchProblem):
    def __init__(self, N, initial_state=None):
        self.N = N

        if initial_state is None:
            initial_state = tuple(self._set_state())
        else:
            is_valid = self._is_valid(initial_state)
            if not is_valid:
                raise ValueError("Invalid initial state!")

        super().__init__(initial_state)

    def __str__(self):
        return f"N: {self.N}\nstate: {self.state}"

    def _set_state(self):
        print("How do you want to set state?")
        print("1. Set state manually")
        print("2. Set state randomly")
        state_choice = int(input("Enter Selection: "))

        if state_choice == 1:
            print("Enter state: ")
            state = input()
            try:
                return [int(s) for s in state]
            except ValueError:
                print("Invalid state input. Please enter integers only.")
                return self._set_state()
        elif state_choice == 2:
            state = self.generate_random_state()
            return state
        else:
            print("Invalid Selection! Please input again...")
            return self._set_state()

    def generate_random_state(self):
        return [random.randint(0, self.N - 1) for _ in range(self.N)]

    def _is_valid(self, state):
        if len(state) != self.N:
            print("Invalid state input. Error Code 1")
            return False
        elif not all(str(s).isdigit() for s in state):
            print("Invalid state input. Error Code 2")
            return False
        elif not all(1 <= int(s) <= self.N for s in state):
            print("Invalid state input. Error Code 3")
            return False
        else:
            return True

    def _count_attacking_pairs(self, state):
        num_pairs = 0

        for i in range(self.N):
            for j in range(i + 1, self.N):
                if state[i] == state[j]:
                    num_pairs += 1

                if abs(state[i] - state[j]) == abs(i - j):
                    num_pairs += 1

        return num_pairs

    def actions(self, state):
        possible_actions = []

        # Iterate through each column to find possible moves for the queen in that column
        for col in range(self.N):
            row = state[col]  # Get the row position of the queen in the current column

            # Generate possible moves within the column for the queen
            for new_row in range(1, self.N + 1):
                if new_row != row:  # Ensure it's a different row than the current one
                    new_state = list(state)  # Create a copy of the current state
                    new_state[col] = new_row  # Move the queen to the new row in the same column
                    possible_actions.append((tuple(new_state), 1))  # Add the new state with a cost of 1

        return possible_actions

    def result(self, state, action):
        new_state, _ = action
        return new_state

    def is_goal(self, state):
        num_attacking_pairs = self._count_attacking_pairs(state)
        return num_attacking_pairs == 0

    def heuristic(self, state):
        num_attacking_pairs = self._count_attacking_pairs(state)
        min_moves = 0
        for i in range(self.N):
            for j in range(i + 1, self.N):
                if abs(state[i] - state[j]) == abs(i - j):
                    min_moves = max(min_moves, abs(state[i] - state[j]))
        return num_attacking_pairs + min_moves



def test_algorithms():
    n_queens_problem = NQueens(4)

    algorithms = [
        breadth_first,
        uniform_cost,
        iterative_limited_depth_first,
        greedy,
        depth_first,
        limited_depth_first,
        astar,
    ]

    for algorithm in algorithms:
        if callable(algorithm):
            algorithm_name = algorithm.__name__
        else:
            algorithm_name = type(algorithm).__name__

        print(f"\nTesting {algorithm_name}:\n")

        starttime = datetime.now()
        if algorithm == depth_first:
            result = depth_first(n_queens_problem, graph_search=True)
        elif algorithm == astar:
            viewer = WebViewer()
            result = astar(n_queens_problem, graph_search=True, viewer=viewer)
        elif algorithm == limited_depth_first:
            result = limited_depth_first(n_queens_problem, depth_limit=100, graph_search=True)
        else:
            result = algorithm(n_queens_problem, graph_search=True)
        endtime = datetime.now()

        print("Resulting state:", result.state if hasattr(result, 'state') else result)
        print("Resulting path:", result.path() if hasattr(result, 'path') else None)
        print("Cost of the solution:", result.cost if hasattr(result, 'cost') else None)

        if hasattr(result, 'stats') and result.stats:
            if isinstance(result.stats, dict):
                print("Viewer statistics:", result.stats)
            else:
                print("Viewer statistics:", result.stats.__dict__)

        print("Time taken:", endtime - starttime)

test_algorithms()

How do you want to set state?
1. Set state manually
2. Set state randomly
Enter state: 

Testing breadth_first:

Resulting state: (2, 4, 1, 3)
Resulting path: [(None, (1, 4, 3, 2)), (((2, 4, 3, 2), 1), (2, 4, 3, 2)), (((2, 4, 1, 2), 1), (2, 4, 1, 2)), (((2, 4, 1, 3), 1), (2, 4, 1, 3))]
Cost of the solution: 3
Time taken: 0:00:00.006196

Testing uniform_cost:

Resulting state: (2, 4, 1, 3)
Resulting path: [(None, (1, 4, 3, 2)), (((2, 4, 3, 2), 1), (2, 4, 3, 2)), (((2, 4, 1, 2), 1), (2, 4, 1, 2)), (((2, 4, 1, 3), 1), (2, 4, 1, 3))]
Cost of the solution: 3
Time taken: 0:00:00.013841

Testing iterative_limited_depth_first:

Resulting state: (3, 1, 4, 2)
Resulting path: [(None, (1, 4, 3, 2)), (((1, 4, 4, 2), 1), (1, 4, 4, 2)), (((1, 1, 4, 2), 1), (1, 1, 4, 2)), (((3, 1, 4, 2), 1), (3, 1, 4, 2))]
Cost of the solution: 3
Time taken: 0:00:00.001042

Testing greedy:

Resulting state: (2, 4, 1, 3)
Resulting path: [(None, (1, 4, 3, 2)), (((1, 4, 2, 2), 1), (1, 4, 2, 2)), (((1, 4, 2, 3), 1), (1, 4

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://10.103.164.1:8000
Press CTRL+C to quit


KeyboardInterrupt: 

AI PA2

In [25]:
from datetime import datetime
from simpleai.search import SearchProblem, hill_climbing, hill_climbing_random_restarts, genetic
from simpleai.search.viewers import WebViewer
import random
import time  # unutulan import

class NQueens(SearchProblem):
    def __init__(self, N, initial_state=None):
        """Class constructor that initializes the attributes."""
        self.N = N  # Determines the number of queens and board size

        if initial_state is None:
            initial_state = tuple(self._set_state())
        else:
            is_valid = self._is_valid(initial_state)
            if not is_valid:
                raise ValueError("Invalid initial state!")

        super().__init__(initial_state)

    def __str__(self):
        """Returns a formatted string that represents the instance."""
        return f"N: {self.N}\nstate: {self.state}"

    def _set_state(self):
        """Sets the instance attribute state by displaying a menu to the user."""
        print("How do you want to set state?")
        print("1. Set state manually")
        print("2. Set state randomly")
        state_choice = int(input("Enter Selection: "))

        if state_choice == 1:
            print("Enter state: ")
            state = input()
            is_valid = self._is_valid(state)
            if is_valid:
                return [int(s) for s in state]
            else:
                print("Invalid state! Please try again...")
                return self._set_state()
        elif state_choice == 2:
            state = self.generate_random_state()
            return state
        else:
            print("Invalid Selection! Please input again...")
            return self._set_state()

    def generate_random_state(self):
        """Generates and returns a valid random state."""
        state = [random.randint(1, self.N) for _ in range(self.N)]
        return state

    def _is_valid(self, state):
        """Internal function that takes a state as input and returns if this is a valid state or not."""
        if len(state) != self.N:  # State length must be equal to N
            print("Invalid state input. Error Code 1")
            return False
        elif not all(s.isdigit() for s in state):  # State must be composed of digits
            print("Invalid state input. Error Code 2")
            return False
        elif not all(1 <= int(s) <= self.N for s in state):
            # The included number of a state must be between 1 and N
            print("Invalid state input. Error Code 3")
            return False
        else:
            return True

    def _count_attacking_pairs(self, state):
        """Primary function of this class. Returns the number of attacking pairs in the given state board."""
        num_pairs = 0

        for i in range(self.N):  # For each column
            for j in range(i + 1, self.N):  # For each column after the current column
                # Column attack check
                if state[i] == state[j]:  # If the queens are in the same row
                    num_pairs += 1  # Add 1 to the number of attacking pairs

                # Diagonal attack check (both directions)
                if abs(state[i] - state[j]) == abs(i - j):  # If the queens are in the same diagonal
                    num_pairs += 1  # Add 1 to the number of attacking pairs

        return num_pairs

    def actions(self, state):
        possible_actions = []

        # Iterate through each column to find possible moves for the queen in that column
        for col in range(self.N):
            row = state[col]  # Get the row position of the queen in the current column

            # Generate possible moves within the column for the queen
            for new_row in range(1, self.N + 1):
                if new_row != row:  # Ensure it's a different row than the current one
                    new_state = list(state)  # Create a copy of the current state
                    new_state[col] = new_row  # Move the queen to the new row in the same column
                    possible_actions.append((tuple(new_state), 1))  # Add the new state with a cost of 1

        return possible_actions

    def result(self, state, action):
        """Computes and returns the resulting new state when the given action is performed from the given state."""
        new_state, _ = action  # Extract the new state from the action
        return new_state

    def is_goal(self, state):
        """Returns whether the state given as a parameter is a goal state."""
        num_attacking_pairs = self._count_attacking_pairs(state)
        return num_attacking_pairs == 0

    def heuristic(self, state):
        """Returns the estimated solution cost from the given state to the goal state."""
        num_attacking_pairs = self._count_attacking_pairs(state)

        # Calculate the minimum number of moves required to eliminate all attacks
        min_moves = 0
        for i in range(self.N):
            for j in range(i + 1, self.N):
                if abs(state[i] - state[j]) == abs(i - j):
                    min_moves = max(min_moves, abs(state[i] - state[j]))

        # Return the heuristic value
        return num_attacking_pairs + min_moves

    def value(self, state):
        """Returns the number of non-attacking pairs of queens for the given state."""
        num_attacking_pairs = self._count_attacking_pairs(state)
        return self.N * (self.N - 1) // 2 - num_attacking_pairs

    def crossover(self, state1, state2):
        """Crossover method for genetic search."""
        crossover_point = random.randint(1, self.N - 1)
        new_state = state1[:crossover_point] + state2[crossover_point:]
        return new_state

    def mutate(self, state):
        """Mutation method for genetic search."""
        mutated_state = list(state)
        index_to_mutate = random.randint(0, self.N - 1)
        mutated_state[index_to_mutate] = random.randint(1, self.N)
        return mutated_state

def hill_climbing_restart_lambda(problem):
    return hill_climbing_random_restarts(problem, 100)

def test_algorithms():
    # Create an instance of NQueens problem with N=8 for testing
    n_queens_problem = NQueens(4)

    # Algorithms to test
    algorithms = [
        hill_climbing,
        hill_climbing_restart_lambda,  # Use the named function instead of lambda
        genetic,  # Directly include the genetic function without calling it
    ]

    for algorithm in algorithms:
        if callable(algorithm):  # Check if algorithm is a function
            algorithm_name = algorithm.__name__
        else:  # If not a function, it's an instance with a class
            algorithm_name = algorithm.__class__.__name__

        print(f"\nTesting {algorithm_name}:\n")
        start_time = datetime.now()

        if algorithm in [hill_climbing, hill_climbing_restart_lambda]:
            result = algorithm(n_queens_problem)  # Adjust parameters directly in the function call
        elif algorithm == genetic:
            time_limit = 5  # Set a time limit for genetic algorithm (in seconds)
            start_time_genetic = time.time()
            result = None
            while (time.time() - start_time_genetic) < time_limit:
                result = algorithm(n_queens_problem, population_size=50)
            # You can add additional parameters as needed
        else:
            result = algorithm(n_queens_problem)

        end_time = datetime.now()
        result.elapsed_time = end_time - start_time
        print("Resulting state:", result.state)
        print("Resulting path:", result.path())
        print("Cost of the solution:", result.cost)

        # Print time taken
        print("Time taken:", result.elapsed_time)

# Call the testing function
test_algorithms()

How do you want to set state?
1. Set state manually
2. Set state randomly
Enter state: 

Testing hill_climbing:

Resulting state: (1, 1, 4, 2)
Resulting path: [(((1, 1, 4, 2), 1), (1, 1, 4, 2))]
Cost of the solution: 3
Time taken: 0:00:00.001047

Testing hill_climbing_restart_lambda:

Resulting state: (3, 1, 4, 2)
Resulting path: [(((3, 1, 4, 2), 1), (3, 1, 4, 2))]
Cost of the solution: 2
Time taken: 0:00:00.015647

Testing genetic:
Resulting state: [3, 1, 4, 4]
Resulting path: [('crossover', [3, 1, 4, 4])]
Cost of the solution: 0
Time taken: 0:00:05.000862
